In [1]:
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
base_model = tf.keras.applications.VGG16(include_top=False, input_shape=(32, 32, 3))

In [4]:
# freeze layers
for layer in base_model.layers:
    layer.trainable = False

In [5]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=base_model.output)
x_train_features = model.predict(x_train)
x_test_features = model.predict(x_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 115s 74ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 73ms/step


In [6]:
x_train_features = x_train_features.reshape((x_train_features.shape[0], -1))
x_test_features = x_test_features.reshape((x_test_features.shape[0], -1))

In [7]:
results = {}

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [9]:
# random forest classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(x_train_features, y_train.ravel())
y_pred_rf = rf_classifier.predict(x_test_features)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
results['Random Forest'] = accuracy_rf

In [10]:
# logistic regression 
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train_features, y_train.ravel())
y_pred_lr = lr.predict(x_test_features)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
results['Logistic Regression'] = accuracy_lr

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
cnn_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 139s 88ms/step - accuracy: 0.4663 - loss: 1.5118 - val_accuracy: 0.5620 - val_loss: 1.2328
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 140s 90ms/step - accuracy: 0.5932 - loss: 1.1591 - val_accuracy: 0.5828 - val_loss: 1.1968
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 141s 90ms/step - accuracy: 0.6156 - loss: 1.0975 - val_accuracy: 0.5866 - val_loss: 1.1682
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 138s 88ms/step - accuracy: 0.6373 - loss: 1.0364 - val_accuracy: 0.6012 - val_loss: 1.1486
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 138s 88ms/step - accuracy: 0.6489 - loss: 0.9883 - val_accuracy: 0.6088 - val_loss: 1.1372


In [12]:
# Evaluate the CNN model
cnn_accuracy = cnn_model.evaluate(x_test, y_test, verbose=0)[1]
results['Pre-trained CNN'] = cnn_accuracy

In [13]:
for classifier, accuracy in results.items():
    print(f'{classifier} accuracy: {accuracy:.4f}')

Random Forest accuracy: 0.5308
Logistic Regression accuracy: 0.5919
Pre-trained CNN accuracy: 0.6088
